In [1]:
#!/usr/bin/env python3
from ZOOMUTILS import ZoomUtils
import pandas as pd
from datetime import datetime
from dateutil import tz
#from ics import Calendar, Event
from icalendar import Calendar, Event
from icalendar import vDatetime

conn = ZoomUtils.create_connection('SynapseS_tempo_20220131.db')
cur= conn.cursor()
sql = '''SELECT distinct a.code_ue, a.date, a.heuredeb, a.heurefin, a.intitule_occur, a.salle, e.email, 
CASE 
     WHEN INSTR(a.intervenants,'$$') > 0 THEN 
     substring(a.intervenants, 1, INSTR(a.intervenants,'$$')-1)
     ELSE a.intervenants
END intervenants
FROM agenda as a left join email_enseignants as e on (e.intervenants = a.intervenants) and (a.code_ue = e.code_ue)
WHERE substring(lower(salle),7) in ('arago','becquerel','curie','faurre','grégory','lagarrigue','painlevé','poincaré','poisson','sauvy');
 '''
cur.execute(sql)

# Création du DataFrame
columns = [desc[0] for desc in cur.description]
data = cur.fetchall()
df = pd.DataFrame(list(data), columns=columns)

In [2]:
# Liste des emails vides et intervenants non vides
for index, row in df[df.email.isnull() & (df.intervenants.values != '')].iterrows():
    # mise à  jour bidon
    # ici faire un appel LDAP
    df.at[index,'email'] = f'{index}@tto.fr'
    

In [11]:
# on travaille sur les programmations par amphi
for amphi in df['salle'].unique():
    print(amphi)
    sdf = df[df.salle == amphi]
    
    # on crée une collection de rdv
    cal = Calendar()
    cal.add('prodid', '-//Polytechnique//SynapseS Miris//')
    cal.add('version', '1.0')
    for index, row in sdf.iterrows():
        """
        #
        # version ical
        #
        c = Calendar()
        e = Event()
        datedeb = row.date + ' ' + row.heuredeb
        datedeb = datetime.strptime(datedeb,'%Y-%m-%d %H:%M:%S').replace(tzinfo=tz.gettz('Europe/Paris'))
        datedeb = datedeb.astimezone(tz.tzutc())
        
        datefin = row.date + ' ' + row.heurefin
        datefin = datetime.strptime(datefin,'%Y-%m-%d %H:%M:%S').replace(tzinfo=tz.gettz('Europe/Paris'))
        datefin = datefin.astimezone(tz.tzutc())
        
        e.name = row.intitule_occur
        e.begin = datedeb.strftime('%Y-%m-%d %H:%M:%S') # Updated this to use date_format
        e.end = datefin.strftime('%Y-%m-%d %H:%M:%S')
        
        desc = {}
        desc['profile'] = "camera-ordinateur"
        desc['title'] = f"{row.code_ue} {row.intitule_occur}"
        if row.intervenants != '':
            desc['speaker'] = row.intervenants
        if row.email != None:
            desc['speaker_email'] = row.email
        e.description = str(desc)

        c.events.add(e)
        c.events
        cs.append(c)
        
        # [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
        with open(amphi + '.ics', 'w') as my_file:
            for c in cs:
                my_file.writelines(c)        
        """
        #
        # version icalendar
        #
        event = Event()

        event.add('summary', f"{row.code_ue} {row.intitule_occur}")
        
        datedeb = row.date + ' ' + row.heuredeb
        datedeb = datetime.strptime(datedeb,'%Y-%m-%d %H:%M:%S').replace(tzinfo=tz.gettz('Europe/Paris'))
        datedeb = datedeb.astimezone(tz.tzutc())
        event.add('dtstart', datedeb)
        
        datefin = row.date + ' ' + row.heurefin
        datefin = datetime.strptime(datefin,'%Y-%m-%d %H:%M:%S').replace(tzinfo=tz.gettz('Europe/Paris'))
        datefin = datefin.astimezone(tz.tzutc())
        event.add('dtend', datefin)
        
        event.add('dtstamp', datetime.now().astimezone(tz.tzutc()))

        desc = {}
        desc['profile'] = "camera-ordinateur"
        desc['title'] = f"{row.code_ue} {row.intitule_occur}"
        if row.intervenants != '':
            desc['speaker'] = row.intervenants
        if row.email != None:
            desc['speaker_email'] = row.email
        event.add('description', str(desc))
        
        cal.add_component(event)
        
    with open(amphi + '.ics', 'wb') as my_file:
        my_file.write(cal.to_ical())

Amphi Curie
VCALENDAR({'PRODID': vText('b'-//SynapseS Miris//mxm.dk//''), 'VERSION': vText('b'1.0'')}, VEVENT({'SUMMARY': vText('b'BIO/CHI567 Sciences du m\xc3\xa9dicament''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x7ff4d076cc90>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x7ff4d076c650>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x7ff4d06efd50>, 'DESCRIPTION': vText('b"{'profile': 'camera-ordinateur'\\, 'title': 'BIO/CHI567 Sciences du m\xc3\xa9dicament'\\, 'speaker': 'Nay Bastien'\\, 'speaker_email': 'bastien.nay@polytechnique.edu'}"')}), VEVENT({'SUMMARY': vText('b'BIO/CHI567 Sciences du m\xc3\xa9dicament''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x7ff4d06effd0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x7ff4d06efa10>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x7ff4d06efc50>, 'DESCRIPTION': vText('b"{'profile': 'camera-ordinateur'\\, 'title': 'BIO/CHI567 Sciences du m\xc3\xa9dicament'\\, 'speaker': 'Nay Bastien'\\, 'speaker_email': 'bastien.nay@p

In [83]:
df

,code_ue,date,heuredeb,heurefin,intitule_occur,salle,email,intervenants
0,BIO/CHI567,2022-01-31,14:00:00,16:00:00,Sciences du médicament,Amphi Curie,bastien.nay@polytechnique.edu,Nay Bastien
1,BIO/CHI567,2022-01-31,16:15:00,18:15:00,Sciences du médicament,Amphi Curie,bastien.nay@polytechnique.edu,Nay Bastien
2,BIO451,2022-01-31,13:30:00,15:00:00,Biologie cellulaire et développement,Amphi Arago,ndavid@biologie.ens.fr,David Nicolas
3,BIO562,2022-01-31,08:30:00,10:30:00,Biologie des systèmes moléculaires,Amphi Lagarrigue,meriem.elkaroui@ed.ac.uk,El Karoui Meriem
4,BIO562,2022-01-31,10:45:00,12:45:00,Biologie des systèmes moléculaires,Amphi Lagarrigue,meriem.elkaroui@ed.ac.uk,El Karoui Meriem
...,...,...,...,...,...,...,...,...
116,PHY580,2022-02-01,13:30:00,18:30:00,"Quantum information : entanglement, control an...",Amphi Painlevé,None,Ayral Thomas
117,PHY582,2022-02-01,13:30:00,18:30:00,Current Trends in Materials Science,Amphi Sauvy,None,Biermann Silke
118,PHY586,2022-02-01,13:30:00,18:30:00,Technologie des réacteurs nucléaires et cycle ...,Amphi Poisson,None,Boullis Bernard
119,PHY661I,2022-01-31,08:30:00,10:30:00,Smart grid for renewable energy,Amphi Poisson,None,Badosa Jordi


In [87]:
df[df.email.isnull() & (df.intervenants.values != '')]

,code_ue,date,heuredeb,heurefin,intitule_occur,salle,email,intervenants


In [80]:
df

,code_ue,date,heuredeb,heurefin,intitule_occur,salle,email,intervenants
0,BIO/CHI567,2022-01-31,14:00:00,16:00:00,Sciences du médicament,Amphi Curie,bastien.nay@polytechnique.edu,Nay Bastien
1,BIO/CHI567,2022-01-31,16:15:00,18:15:00,Sciences du médicament,Amphi Curie,bastien.nay@polytechnique.edu,Nay Bastien
2,BIO451,2022-01-31,13:30:00,15:00:00,Biologie cellulaire et développement,Amphi Arago,ndavid@biologie.ens.fr,David Nicolas
3,BIO562,2022-01-31,08:30:00,10:30:00,Biologie des systèmes moléculaires,Amphi Lagarrigue,meriem.elkaroui@ed.ac.uk,El Karoui Meriem
4,BIO562,2022-01-31,10:45:00,12:45:00,Biologie des systèmes moléculaires,Amphi Lagarrigue,meriem.elkaroui@ed.ac.uk,El Karoui Meriem
...,...,...,...,...,...,...,...,...
116,PHY580,2022-02-01,13:30:00,18:30:00,"Quantum information : entanglement, control an...",Amphi Painlevé,116@tto.fr,Ayral Thomas
117,PHY582,2022-02-01,13:30:00,18:30:00,Current Trends in Materials Science,Amphi Sauvy,117@tto.fr,Biermann Silke
118,PHY586,2022-02-01,13:30:00,18:30:00,Technologie des réacteurs nucléaires et cycle ...,Amphi Poisson,118@tto.fr,Boullis Bernard
119,PHY661I,2022-01-31,08:30:00,10:30:00,Smart grid for renewable energy,Amphi Poisson,119@tto.fr,Badosa Jordi
